In [1]:
import numpy as np
import pandas as pd

# Assignment 1: Groupby

Can you return a table containing the top 10 stores by total transactions in the data?

Make sure they’re sorted from highest to lowest.

Thanks!

In [2]:
# Read in transactions data -- parse dates specified here for help with later problem

transactions = pd.read_csv("../retail/transactions.csv", parse_dates=["date"])

transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [3]:
transactions.info()
tx_by_store = pd.DataFrame(transactions.groupby('store_nbr')['transactions'].sum())
tx_by_store.sort_values('transactions', ascending=False).head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


,transactions
store_nbr,
44,7273093
47,6535810
45,6201115
46,5990113
3,5366350
48,5107785
8,4637971
49,4574103
50,4384444


# Assignment 2: Groupby Multiple Columns

Can you get me the total transactions by store and month? 

Sort the table from first month to last, then by highest transactions to lowest within each month. 


In [4]:
# helper code to extract month date part from date column

transactions["month"] = transactions["date"].dt.month

transactions.head()

result = transactions.groupby(['store_nbr', 'month'])[['transactions']].sum()
result.index

MultiIndex([( 1,  1),
            ( 1,  2),
            ( 1,  3),
            ( 1,  4),
            ( 1,  5),
            ( 1,  6),
            ( 1,  7),
            ( 1,  8),
            ( 1,  9),
            ( 1, 10),
            ...
            (54,  3),
            (54,  4),
            (54,  5),
            (54,  6),
            (54,  7),
            (54,  8),
            (54,  9),
            (54, 10),
            (54, 11),
            (54, 12)],
           names=['store_nbr', 'month'], length=641)

In [5]:
result.sort_values(['month', 'transactions'], ascending=[True, False])

,,transactions
store_nbr,month,
44,1,628438
47,1,568824
45,1,538370
46,1,522763
3,1,463260
...,...,...
32,12,86167
21,12,84128
42,12,76741


# Assignment 3: Multi-Index DataFrames


Can you help me access rows and columns with multiple indices? I’ve been struggling with multi-index DataFrames.

Access:
* Grab Store 3, Month 1
* Then, select the column storing the mean of transactions

Fix:
* Drop the outer layer of the column Index
* Reset the row index so it is the default integer index

In [6]:
# Ross' grouped DataFrame code, run this first

grouped = (
    transactions.groupby(["store_nbr", "month"])
    .agg({"transactions": ["sum", "mean"]})
    .sort_values(by=["month", ("transactions", "sum")], ascending=[True, False])
)

In [7]:
# df grouped by store and month with two aggregation columns

grouped.head()

transactions             
                         sum         mean
store_nbr month                          
44        1           628438  4246.202703
47        1           568824  3843.405405
45        1           538370  3637.635135
46        1           522763  3532.182432
3         1           463260  3151.428571

In [8]:
grouped.loc[(3, 1), :].transactions['mean']

np.float64(3151.4285714285716)

In [9]:
grouped.droplevel(0, axis=1)

,,sum,mean
store_nbr,month,,
44,1,628438,4246.202703
47,1,568824,3843.405405
45,1,538370,3637.635135
46,1,522763,3532.182432
3,1,463260,3151.428571
...,...,...,...
32,12,86167,718.058333
21,12,84128,1402.133333
42,12,76741,1279.016667


In [10]:
grouped.reset_index()

store_nbr month transactions             
                             sum         mean
0          44     1       628438  4246.202703
1          47     1       568824  3843.405405
2          45     1       538370  3637.635135
3          46     1       522763  3532.182432
4           3     1       463260  3151.428571
..        ...   ...          ...          ...
636        32    12        86167   718.058333
637        21    12        84128  1402.133333
638        42    12        76741  1279.016667
639        29    12        76627  1277.116667
640        22    12        50650   844.166667

[641 rows x 4 columns]

# Assignment 4: The Agg Method

Calculate the mean of target met by store, and the sum of bonuses to be paid to each store.

Sort them by highest to lowest bonus payout.

Then, do the same for day of week and month.

In [11]:
# Recreate table from section 3

transactions = transactions.assign(
    target_pct=transactions["transactions"] / 2500,
    met_target=(transactions["transactions"] / 2500) >= 1,
    bonus_payable=((transactions["transactions"] / 2500) >= 1) * 100,
    month=transactions.date.dt.month,
    day_of_week=transactions.date.dt.dayofweek,
)

transactions.head()

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week
0,2013-01-01,25,770,1,0.3080,False,0,1
1,2013-01-02,1,2111,1,0.8444,False,0,2
2,2013-01-02,2,2358,1,0.9432,False,0,2
3,2013-01-02,3,3487,1,1.3948,True,100,2
4,2013-01-02,4,1922,1,0.7688,False,0,2


In [12]:
(transactions
.groupby('store_nbr', as_index=False)
.agg(target_met_mean=('met_target', 'mean'), bonus_sum=('bonus_payable', 'sum'))
.sort_values('bonus_sum', ascending=False))

,store_nbr,target_met_mean,bonus_sum
46,47,0.999404,167600
43,44,0.998807,167500
2,3,0.998210,167300
44,45,0.997615,167300
45,46,0.989267,165900
7,8,0.888425,148900
47,48,0.690519,115800
48,49,0.637448,106900
49,50,0.453190,76000
10,11,0.296539,49700


In [13]:
(transactions
.groupby('month', as_index=False)
.agg(target_met_mean=('met_target', 'mean'), bonus_sum=('bonus_payable', 'sum'))
.sort_values('bonus_sum', ascending=False)
)

,month,target_met_mean,bonus_sum
11,12,0.255640,154100
4,5,0.170792,131800
2,3,0.169461,130400
3,4,0.174469,129700
6,7,0.162486,126300
1,2,0.174230,121700
5,6,0.161706,121700
7,8,0.174189,120800
0,1,0.163723,119600
10,11,0.163943,98300


In [14]:
(transactions
.groupby('day_of_week', as_index=False)
.agg(target_met_mean=('met_target', 'mean'), bonus_sum=('bonus_payable', 'sum'))
.sort_values('bonus_sum', ascending=False)
)

,day_of_week,target_met_mean,bonus_sum
5,5,0.222204,266400
6,6,0.204001,241700
4,4,0.179007,213000
0,0,0.160214,191600
2,2,0.160572,191000
1,1,0.146299,175500
3,3,0.142077,169100


# Assignment 4: Transform

Calculate the mean of transactions by store number and day of week while keeping row numbers. 

Then compare the performance of each row to its day of week average. (difference between transactions and daily avg)

In [15]:
transactions = transactions.assign(mean_sales = (transactions
.groupby(['store_nbr', 'day_of_week'], as_index=False)['transactions']
.transform('mean')))

transactions.head()

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week,mean_sales
0,2013-01-01,25,770,1,0.3080,False,0,1,740.245690
1,2013-01-02,1,2111,1,0.8444,False,0,2,1870.782427
2,2013-01-02,2,2358,1,0.9432,False,0,2,1952.652720
3,2013-01-02,3,3487,1,1.3948,True,100,2,3142.682008
4,2013-01-02,4,1922,1,0.7688,False,0,2,1499.569038


In [16]:
transactions.assign(performance_delta=(transactions['transactions'] - transactions['mean_sales']))

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week,mean_sales,performance_delta
0,2013-01-01,25,770,1,0.3080,False,0,1,740.245690,29.754310
1,2013-01-02,1,2111,1,0.8444,False,0,2,1870.782427,240.217573
2,2013-01-02,2,2358,1,0.9432,False,0,2,1952.652720,405.347280
3,2013-01-02,3,3487,1,1.3948,True,100,2,3142.682008,344.317992
4,2013-01-02,4,1922,1,0.7688,False,0,2,1499.569038,422.430962
...,...,...,...,...,...,...,...,...,...,...
83483,2017-08-15,50,2804,8,1.1216,True,100,1,2342.410788,461.589212
83484,2017-08-15,51,1573,8,0.6292,False,0,1,1548.448133,24.551867
83485,2017-08-15,52,2255,8,0.9020,False,0,1,1892.588235,362.411765
83486,2017-08-15,53,932,8,0.3728,False,0,1,877.214286,54.785714


# Assignment 5: Pivot

Pivot transactions with store number as index, columns day of week, with the sum of bonus payable as cells.

Filter to stores that had a non-zero bonus payable and create a heatmap.

Then unpivot (melt) the table so we have one row for each store and day of the week with the corresponding total owed. 


In [17]:
# Use transactions table (ok if includes columns from assignment 4 or not)

transactions.head()

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week,mean_sales
0,2013-01-01,25,770,1,0.3080,False,0,1,740.245690
1,2013-01-02,1,2111,1,0.8444,False,0,2,1870.782427
2,2013-01-02,2,2358,1,0.9432,False,0,2,1952.652720
3,2013-01-02,3,3487,1,1.3948,True,100,2,3142.682008
4,2013-01-02,4,1922,1,0.7688,False,0,2,1499.569038


In [29]:
import matplotlib
import pandas as pd

(transactions
.query('bonus_payable != 0')
.pivot_table(index='store_nbr', columns='day_of_week', values='bonus_payable', aggfunc='sum')
.iloc[:10]
.style
.background_gradient(cmap="RdYlGn", axis=1))

day_of_week,0,1,2,3,4,5,6
store_nbr,,,,,,,
1,200.000000,300.000000,300.000000,200.000000,100.000000,nan,nan
2,300.000000,600.000000,500.000000,400.000000,400.000000,500.000000,200.000000
3,24000.000000,23900.000000,23900.000000,23900.000000,23900.000000,24000.000000,23700.000000
4,200.000000,300.000000,300.000000,200.000000,100.000000,200.000000,nan
5,200.000000,300.000000,300.000000,100.000000,100.000000,100.000000,nan
6,400.000000,500.000000,500.000000,300.000000,200.000000,900.000000,300.000000
7,200.000000,300.000000,300.000000,200.000000,100.000000,100.000000,nan
8,22000.000000,18800.000000,23800.000000,18000.000000,22900.000000,23400.000000,20000.000000
9,1200.000000,800.000000,800.000000,700.000000,400.000000,7900.000000,5100.000000


In [36]:
(transactions
.query('bonus_payable != 0')
.melt(id_vars=['store_nbr'], value_vars='bonus_payable', var_name='bonus_payable', value_name='total_owed'))

,store_nbr,bonus_payable,total_owed
0,3,bonus_payable,100
1,8,bonus_payable,100
2,9,bonus_payable,100
3,11,bonus_payable,100
4,24,bonus_payable,100
...,...,...,...
14478,46,bonus_payable,100
14479,47,bonus_payable,100
14480,48,bonus_payable,100
14481,49,bonus_payable,100
